In [1]:
#!pip install pygoogletranslation

^C


In [13]:
from pygoogletranslation import Translator
translator = Translator()
source = "Good Morning"
translation = translator.translate(source, dest='ta').text

In [6]:
import evaluate
metric = evaluate.load("sacrebleu")

In [10]:
result = metric.compute(predictions=translation, references=source) #BLEU score for provided input and references
result = {"bleu": result["score"]}
print(result)

{'bleu': 0.0}


In [27]:
#Load in common data (OPUS set), already separated into training and testing data, prior to tokenisation (for Marian and T5) - just load the test data
from datasets import load_dataset
booksTest = load_dataset("ethansimrm/OpusTest")

Found cached dataset parquet (C:/Users/ethan/.cache/huggingface/datasets/ethansimrm___parquet/ethansimrm--OpusTest-e350203b885f5992/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [28]:
booksTest["train"][0]

{'id': '51354',
 'translation': {'en': 'The day following passed with a new sweetness.',
  'fr': 'La journée du lendemain se passa dans une douceur nouvelle.'}}

In [50]:
booksTestFlattened = booksTest.flatten()
source = booksTestFlattened["train"]["translation.en"]
target = booksTestFlattened["train"]["translation.fr"]

In [73]:
from tqdm import tqdm
from time import sleep
#Query only source strings from the test set and feed into Translator, gathering outputs in a list, then BLEU over all of them.
#ISSUE: Google Translate throttles access. We can do resampling of 200 (when throttling starts) e.g., 100 times, perhaps? Then we compute average BLEU across all.
preds = []
counter = 0
for s in tqdm(source):
    t = translator.translate(s, dest = 'fr').text
    preds.append(t)
    if ((counter != 0) & (counter % 200 == 0)):
        sleep(1)
    counter += 1

  1%|          | 244/25417 [00:37<1:04:27,  6.51it/s]


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
result = metric.compute(predictions=translation, references=source) #BLEU score for provided input and references
result = {"bleu": result["score"]}